# html을 통째로 한 번에 추출하는 지자체들

- 처음은 시간이 급해서 일단 전체 document를 통째로 뽑았는데, 나중에 특정 파트를 찾는 게 DB를 더 효율적으로 만들게 할 수 있을 것 같다.

In [22]:
import datetime as dt
import numpy as np
import pandas as pd
from pathlib import Path
import time
from tqdm import tqdm
from selenium import webdriver

In [23]:
today = str(dt.datetime.today().date()).replace('-','')[2:]
PATH_DATA = Path.cwd() / 'tables' / today

chromedriver = '/Users/junseoklee/Applications/chromedriver'
options = webdriver.ChromeOptions()
options.add_argument('headless')
driver = webdriver.Chrome(chromedriver, options = options) 
driver.set_window_size(2000,1000)

In [24]:
one_shot_urls = {
    '1080': 'https://www.gangnam.go.kr/',
    '1081': 'https://www.gangdong.go.kr/site/contents/corona/',
    '1082': 'http://www.gangbuk.go.kr/intro_gb.jsp',
    '1086': 'https://www.guro.go.kr/corona2.jsp',
    '1089': 'http://www.dobong.go.kr/Contents.asp?code=10007909',
    '1091': 'https://www.dongjak.go.kr/',
    '1092': 'https://www.mapo.go.kr/site/main/content/mapo031004',
    '1093': 'https://www.sdm.go.kr/news/corona19/coronaInfo.do?mode=view&sdmBoardSeq=236109',
    '1094': 'https://www.seocho.go.kr/html/notice/main.jsp#con12',
    '1095': 'http://www.sd.go.kr/sd/intro.do',
    '1096': 'http://www.sb.go.kr/PageLink.do?link=forward:use/PageContent.do&tempParam1=&menuNo=05000000&subMenuNo=05140000&thirdMenuNo=05141300&fourthMenuNo=',
    '1097': 'http://songpa-co.asadal.com/route/table.htm?param=home',
    '1100': 'https://www.yongsan.go.kr/pms/contents/contents.do?contseqn=2327&sitecdv=S0000100&menucdv=08020300&decorator=pmsweb',
    '1101': 'https://www.ep.go.kr/CmsWeb/viewPage.req?idx=PG0000004916',
    '1102': 'https://www.jongno.go.kr/Main.do?menuId=400516&menuNo=400516',
    '1103': 'http://www.junggu.seoul.kr/content.do?cmsid=14671',
    '1180': 'http://www.gp.go.kr/intro_gp.jsp',
    '1184': 'https://www.gjcity.go.kr/corona_index.jsp',
    '1185': 'https://www.guri.go.kr/cms/content/view/6410',
    '1187': 'https://www.gimpo.go.kr/portal/contents.do?key=4845#n',
    '1188': 'https://www.nyj.go.kr/',
    '1189': 'http://www.ddc.go.kr/',
    '1190': 'https://www.bucheon.go.kr/site/program/board/basicboard/view?menuid=148004001007&pagesize=10&boardtypeid=27426&encid=Uv3M/oRg2uBizyQm+L2uSw==',
    '1193': 'https://www.siheung.go.kr/corona_policy.jsp',
    '1195': 'https://www.anseong.go.kr/corona_index.jsp',
    '1197': 'https://www.yangju.go.kr/intro.jsp',
    '1199': 'https://www.yeoju.go.kr/corona.jsp',
    '1201': 'https://www.osan.go.kr/corona_index.jsp',
    '1205': 'https://www.icheon.go.kr/portal/contents.do?key=3568',
    '1207': 'https://www.pyeongtaek.go.kr/pyeongtaek/corona_index.jsp',
    '1066': 'https://www.gyeyang.go.kr/open_content/main/open_info/admin/covid/path.jsp',
    '1067': 'http://www.namdong.go.kr/covid-19/index.jsp',
    '1068': 'https://www.icdonggu.go.kr/open_content/main/open_info/covid/path.jsp',
    '1070': 'https://www.icbp.go.kr/main/participation/news/corona_route.jsp',
    '1071': 'http://www.seo.incheon.kr/open_content/main/community/news/corona_route.jsp',
    '1079': 'http://www.junggu.ulsan.kr/board/view.ulsan?boardId=BBS_0000185&startPage=1&dataSid=599080&menuCd=DOM_000000302007012000&contentsSid=2542&cpath=',
    '1080': 'http://www.seo.incheon.kr/open_content/main/community/news/corona_route.jsp',
    '1081': 'http://www.yeonsu.go.kr/covid-19/',
    '1003': 'https://www.boeun.go.kr/intro.jsp',
    '1004': 'http://yd21.go.kr/corona.html',
    '1005': 'https://www.oc.go.kr/intro_oc.jsp',
    '1006': 'https://www.eumseong.go.kr/index_corona.jsp#n',
    '1007': 'http://www.jecheon.go.kr/site/corona01/index.html',
    '1010': 'https://corona.cheongju.go.kr/corona.html',
    '1013': 'http://www.gongju.go.kr/corona.jsp#n',
    '1014': 'http://www.geumsan.go.kr/corona.html#move',
    '1015': 'http://www.nonsan.go.kr/site/corona/index.html#txt',
    '1017': 'http://www.brcn.go.kr/kor/sub04_09_05/covid/route/list.do',
    '1018': 'http://www.buyeo.go.kr/corona.html',
    '1019': 'http://www.seosan.go.kr/intro.jsp',
    '1020': 'https://www.seocheon.go.kr/',
    '1021': 'https://www.asan.go.kr/main/corona/',
    '1022': 'http://www.yesan.go.kr/',
    '1023': 'http://www.cheonan.go.kr/prog/coronaRoute/covid19/sub02_01/list.do?show_ab=A',
    '1024': 'http://www.cheongyang.go.kr/intro_corona.jsp',
    '1026': 'https://www.hongseong.go.kr/intro_corona.jsp',
    '1139': 'http://gbgs.go.kr/programs/coronaMove/coronaMove.do',
    '1141': 'https://www.goryeong.go.kr/intro02.html',
    '1142': 'http://www.gumi.go.kr/portal/contents.do?mId=0610100400',
    '1143': 'https://www.gunwi.go.kr/ko/page.do?mnu_uid=1030&',
    '1145': 'https://www.gbmg.go.kr/portal/contents.do?mId=0604110200',
    '1146': 'https://www.bonghwa.go.kr/open.content/ko/news/corona/corona.info/move/',
    '1148': 'https://sj.go.kr/main.jsp#self',
    '1149': 'https://www.andong.go.kr/portal/contents/corona_index.jsp',
    '1150': 'https://yd.go.kr/?p=65112',
    '1151': 'http://static.yyg.go.kr/COVID.html?build=20200831111601#none',
    '1152': 'http://www.yeongju.go.kr/open_content/main/page.do?mnu_uid=10666',
    '1154': 'http://www.ycg.kr/open.content/health/health.info/corona/today/?i=69653',
    '1156': 'http://www.uljin.go.kr/index.uljin?menuCd=DOM_000000110006003000',
    '1158': 'http://www.cheongdo.go.kr/open.content/health/corona/move/',
    '1161': 'http://www.pohang.go.kr/COVID-19.html',
    '1163': 'https://www.geochang.go.kr/',
    '1164': 'http://www.goseong.go.kr/index.goseong?contentsSid=5814&cpath=',
    '1165': 'http://www.gimhae.go.kr/03360/00023/05877.web',
    '1166': 'http://www.namhae.go.kr/health/corona/View.do?&pageCd=HF0306000000&siteGubun=health',
    '1167': 'https://www.miryang.go.kr/html/dpt/heabus/index.jsp',
    '1168': 'https://www.sacheon.go.kr/intro/01613/03928.web',
    '1169': 'https://www.sancheong.go.kr/intro_sc.jsp#n',
    '1170': 'http://www.yangsan.go.kr/portal/contents/popup/corona/corona19.jsp',
    '1171': 'https://www.uiryeong.go.kr/index.uiryeong#close',
    '1172': 'http://www.jinju.go.kr/05190/05641.web',
    '1173': 'http://www.cng.go.kr/cng-popup/00005548.web',
    '1174': 'https://www.changwon.go.kr/index3.html',
    '1175': 'http://www.tongyeong.go.kr/01174/04510.web',
    '1176': 'http://www.hadong.go.kr/01036/03369.web',
    '1177': 'https://www.haman.go.kr/05960/07001.web',
    '1178': 'http://www.hygn.go.kr/03017/04262.web',
    '1179': 'https://www.hc.go.kr/07370/08804.web',
    '1211': 'https://www.gn.go.kr/',
    '1215': 'http://www.sokcho.go.kr/intro.html',
    '1217': 'http://www.yangyang.go.kr/covid/corona-19.html',
    '1219': 'http://covid19.wonju.go.kr/intro.jsp#',
    '1220': 'http://www.inje.go.kr/corona-19.html',
    '1221': 'http://www.jeongseon.go.kr/intro.html',
    '1222': 'http://www.cwg.go.kr/coronaintro.jsp',
    '1223': 'https://www.chuncheon.go.kr/index.chuncheon?menuCd=DOM_000000599001000000',
    '1224': 'https://www.taebaek.go.kr/intro.jsp',
    '1225': 'https://www.pc.go.kr/corona-19.html',
    '1226': 'https://www.hongcheon.go.kr/corona_intro.jsp',
    '1227': 'http://www.ihc.go.kr/corona.jsp',
    '1228': 'https://www.hsg.go.kr/00001842/00002873.web',
    '1126': 'https://nam.daegu.kr/?menu_id=00203710',
    '1127': 'https://dalseo.daegu.kr/index.do?menu_id=00002262',
    '1128': 'https://www.dalseong.daegu.kr/index.do?menu_id=00002910',
    '1129': 'https://dong.daegu.kr/routelist/indexnew.php',
    '1130': 'https://www.buk.daegu.kr/icms/bbs/selectBoardList.do?bbsId=BBSMSTR_000000001881',
    '1132': 'http://www.suseong.kr/?menu_id=00040470',
    '1133': 'http://www.jung.daegu.kr/new/pages/main/covid_19_flow.html',
    '1105': 'http://www.bsgangseo.go.kr/bsgangseo/bbs/view.do?bIdx=107601&bsgsIdx=21&mId=0401010000',
    '1106': 'http://www.geumjeong.go.kr/index.geumj?contentsSid=5845',
    '1107': 'http://www.gijang.go.kr/health/index.gijang?menuCd=DOM_000000402004004006',
    '1109': 'http://www.bsdonggu.go.kr/index.donggu?contentsSid=1433',
    '1110': 'https://www.dongnae.go.kr/index.dongnae?menuCd=DOM_000000112014000000',
    '1111': 'http://www.busanjin.go.kr/corona-road.html',
    '1112': 'http://www.bsbukgu.go.kr/index.bsbukgu?contentsSid=2317',
    '1113': 'https://www.sasang.go.kr/index.sasang?contentsSid=2407',
    '1114': 'https://www.saha.go.kr/corona.do',
    '1115': 'https://www.bsseogu.go.kr/index.bsseogu?contentsSid=2552',
    '1116': 'http://www.suyeong.go.kr/index.suyeong?contentsSid=2180',
    '1117': 'https://www.yeonje.go.kr/index2.jsp',
    '1118': 'https://www.yeongdo.go.kr/02860/03571.web',
    '1119': 'http://www.bsjunggu.go.kr/board/view.junggu?boardId=BBS_0000001&menuCd=DOM_000000103001001000&paging=ok&startPage=1&dataSid=204929',
    '1120': 'http://www.haeundae.go.kr/health/index.do?menuCd=DOM_000000804005007004',
    '1043': 'http://www.gangjin.go.kr/COVID.html?build=f60f92c6c177b1bbe5f734d439f52732#none',
    '1045': 'http://www.gokseong.go.kr/covid19/',
    '1046': 'http://www.gwangyang.go.kr/index.gwangyang?menuCd=DOM_000000102011009000',
    '1047': 'https://www.gurye.go.kr/kr/subPage.do?menuNo=117015004000',
    '1048': 'http://www.naju.go.kr/COVID.html',
    '1050': 'http://www.mokpo.go.kr/COVID.html?build=20200831165306#none',
    '1052': 'http://www.boseong.go.kr/COVID.html?build=9acabec52d17c7a013c51a29aeb82956#none',
    '1053': 'https://www.suncheon.go.kr/emer1/index.html',
    '1054': 'http://www.shinan.go.kr/home/www/dept_info/health/covid/course/page.wscms',
    '1055': 'http://www.yeosu.go.kr/www/pubinfo/mn01/mn0103/mn010304/yeosu.go',
    '1057': 'https://www.yeongam.go.kr/home/www/info_area/life_safety/life_safety_18/yeongam.go',
    '1058': 'http://static.wando.go.kr/COVID.html?build=9acabec52d17c7a013c51a29aeb82956#none',
    '1059': 'http://jangseong.go.kr/home/www/openinfo/safety/health_25/health_25_04',
    '1061': 'https://www.jindo.go.kr/home/sub.cs?m=742',
} 

In [25]:
df = pd.DataFrame().from_dict(one_shot_urls, orient = 'index').reset_index()
df.columns = ['num', 'url']
df['html'] = ''
df

,num,url,html
0,1080,http://www.seo.incheon.kr/open_content/main/co...,
1,1081,http://www.yeonsu.go.kr/covid-19/,
2,1082,http://www.gangbuk.go.kr/intro_gb.jsp,
3,1086,https://www.guro.go.kr/corona2.jsp,
4,1089,http://www.dobong.go.kr/Contents.asp?code=1000...,
...,...,...,...
130,1055,http://www.yeosu.go.kr/www/pubinfo/mn01/mn0103...,
131,1057,https://www.yeongam.go.kr/home/www/info_area/l...,
132,1058,http://static.wando.go.kr/COVID.html?build=9ac...,
133,1059,http://jangseong.go.kr/home/www/openinfo/safet...,


## 가끔 로딩이 오래 걸리는 사이트가 있어서 중간에 일시적으로 멈추는 경우가 있지만, 잘 참고 기다리면 10분 안에는 끝난다...!

In [26]:
for i, url in tqdm(enumerate(df.url)):
    driver.get(url)
    time.sleep(0.5)
    df.iloc[i,2] = driver.page_source

135it [08:20,  3.71s/it]


In [27]:
df['date'] = today
df.to_csv(PATH_DATA / (today + 'one_shot.csv'), index = False)

In [29]:
driver.quit()